# World Model Experiments: Pixel-based Causal Learning

This notebook runs the initial experiments to test whether the world model learns:
1. **Rules as compressible structure** (low entropy for deterministic transitions)
2. **Stochastic modeling** (matching entropy for chance events)

## Key Test: Entropy Distribution

A well-trained model should show:
- **2048**: Bimodal entropy (deterministic slides + stochastic tile spawns)
- **Othello**: All near-zero entropy (fully deterministic)

In [1]:
# Setup - Run this first
!nvidia-smi -L || echo 'No GPU detected'
import torch
print(f'PyTorch {torch.__version__}, CUDA available: {torch.cuda.is_available()}')

GPU 0: Tesla T4 (UUID: GPU-0534f4d8-2d8f-dc32-b98c-21d4f1c96bb7)
PyTorch 2.9.0+cu126, CUDA available: True


In [2]:
# Mount Google Drive (for saving outputs)
import os, sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    REPO_DIR = '/content/drive/MyDrive/Colab_Notebooks/tg_smn'
    OUT_DIR = '/content/drive/MyDrive/Colab_Notebooks/world_model_outputs'
else:
    REPO_DIR = os.getcwd()
    OUT_DIR = os.path.join(REPO_DIR, 'outputs')

os.makedirs(OUT_DIR, exist_ok=True)
print(f'REPO_DIR: {REPO_DIR}')
print(f'OUT_DIR: {OUT_DIR}')

Mounted at /content/drive
REPO_DIR: /content/drive/MyDrive/Colab_Notebooks/tg_smn
OUT_DIR: /content/drive/MyDrive/Colab_Notebooks/world_model_outputs


In [3]:
# Clone or update repo
import pathlib

if IN_COLAB:
    parent = pathlib.Path(REPO_DIR).parent
    parent.mkdir(parents=True, exist_ok=True)
    %cd {parent}

    if not os.path.exists(REPO_DIR):
        !git clone https://github.com/RespectableGlioma/tg_smn.git
    else:
        %cd {REPO_DIR}
        !git pull

%cd {REPO_DIR}
!ls -la

/content/drive/MyDrive/Colab_Notebooks
/content/drive/MyDrive/Colab_Notebooks/tg_smn
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 12 (delta 7), reused 9 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (12/12), 16.43 KiB | 122.00 KiB/s, done.
From https://github.com/RespectableGlioma/tg_smn
   c848cae..c27ac71  main       -> origin/main
Updating c848cae..c27ac71
Fast-forward
 README.md                                   |   3 +
 world_models/stoch_muzero/train_pixel_v2.py | 480 ++++++++++++++++++++
 world_models/stoch_muzero/train_vq.py       | 590 ++++++++++++++++++++++++
 world_models/stoch_muzero/vq_model.py       | 671 ++++++++++++++++++++++++++++
 4 files changed, 1744 insertions(+)
 create mode 100644 world_models/stoch_muzero/train_pixel_v2.py
 create mode 100644 world_models/stoch_muzero/train_vq.py
 create mode 100644 world_models/stoch_muzero/vq_model.py
/content/drive

In [4]:
# Install dependencies
!pip install -q tqdm numpy

## VQVAE TEST

In [ ]:
! python -m world_models.stoch_muzero.train_vq --game 2048 --train_steps 20000

Training VQ World Model for 2048
  Device: cuda
  Output: outputs_vq_2048
Generating training data...
Generating 2048 trajectories: 100% 2000/2000 [00:15<00:00, 133.08it/s]
  Train: 1800 trajectories
  Val: 200 trajectories
  Model parameters: 1,710,401
Training:   5% 999/20000 [04:50<1:32:12,  3.43it/s, loss=0.5502, trans=0.5351, ent=0.36, perp=2]
Step 1000:
  Val loss: 0.5201
  Val transition: 0.5049
  Val entropy: 0.48 bits
  Entropy stats: mean=0.48, std=0.20
  Frac below 0.5: 76.4%, above 2.0: 0.0%
Training:  10% 1999/20000 [09:46<1:26:45,  3.46it/s, loss=0.1787, trans=0.0999, ent=0.08, perp=12]
Step 2000:
  Val loss: 0.2135
  Val transition: 0.1377
  Val entropy: 0.08 bits
  Entropy stats: mean=0.19, std=0.14
  Frac below 0.5: 97.2%, above 2.0: 0.0%
Training:  15% 2999/20000 [14:41<1:21:32,  3.48it/s, loss=0.4617, trans=0.2725, ent=0.43, perp=30]
Step 3000:
  Val loss: 0.3629
  Val transition: 0.1728
  Val entropy: 0.37 bits
  Entropy stats: mean=0.68, std=0.29
  Frac below 0.5: 

In [ ]:
! python -m world_models.stoch_muzero.train_vq --game othello --train_steps 20000

---
## Experiment 1: Grid-based 2048 (Sanity Check)

First, verify the architecture works with symbolic input before adding perception.

In [ ]:
# Grid-based 2048 training
%cd {REPO_DIR}

!python -m world_models.stoch_muzero.train \
    --game 2048 \
    --collect_episodes 1000 \
    --train_steps 15000 \
    --w_policy 0 --w_value 0 \
    --empty_weight 0.2 \
    --changed_cell_bonus 3.0 \
    --eval_every 3000 \
    --outdir "{OUT_DIR}/grid_2048_sanity"

**Expected Results:**
- cell_acc > 0.85
- changed_acc > 0.65
- entropy_error < 0.5 bits

---
## Experiment 2: Pixel-based 2048 (THE REAL TEST)

This is the key experiment - can the model learn rules from raw pixels?

In [ ]:
# Pixel-based 2048 training
%cd {REPO_DIR}

!python -m world_models.stoch_muzero.train_pixel \
    --game 2048 \
    --collect_episodes 500 \
    --train_steps 10000 \
    --img_size 64 \
    --state_dim 256 \
    --batch_size 32 \
    --eval_every 2000 \
    --outdir "{OUT_DIR}/pixel_2048_baseline"

**Key Metrics to Watch:**
- `recon_loss`: Can it reconstruct observations from latent state?
- `dynamics_loss`: Can it predict future observations?
- `avg_entropy`: Is it learning deterministic structure?

**Expected:**
- Initially high entropy (random predictions)
- Over training, entropy should become **bimodal**:
  - Low entropy cluster (deterministic slides)
  - Higher entropy cluster (stochastic tile spawns)

---
## Experiment 3: Pixel-based Othello (Determinism Test)

Othello has NO randomness - all transitions are deterministic.

In [ ]:
# Pixel-based Othello training
%cd {REPO_DIR}

!python -m world_models.stoch_muzero.train_pixel \
    --game othello \
    --collect_episodes 300 \
    --train_steps 10000 \
    --img_size 64 \
    --state_dim 256 \
    --eval_every 2000 \
    --outdir "{OUT_DIR}/pixel_othello_baseline"

**Expected:**
- ALL predicted entropies should approach 0
- The model should learn that Othello is fully deterministic

---
## Analysis: Compare Entropy Distributions

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

def load_metrics(outdir):
    metrics_path = os.path.join(outdir, 'metrics.json')
    if os.path.exists(metrics_path):
        with open(metrics_path) as f:
            return json.load(f)
    return None

# Load results
results = {
    '2048 (Grid)': load_metrics(f'{OUT_DIR}/grid_2048_sanity'),
    '2048 (Pixel)': load_metrics(f'{OUT_DIR}/pixel_2048_baseline'),
    'Othello (Pixel)': load_metrics(f'{OUT_DIR}/pixel_othello_baseline'),
}

for name, metrics in results.items():
    if metrics:
        print(f'\n{name}:')
        for k, v in metrics.items():
            if isinstance(v, float):
                print(f'  {k}: {v:.4f}')

In [ ]:
# Plot entropy histograms if available
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, (name, outdir) in zip(axes, [
    ('2048 Grid', f'{OUT_DIR}/grid_2048_sanity'),
    ('2048 Pixel', f'{OUT_DIR}/pixel_2048_baseline'),
    ('Othello Pixel', f'{OUT_DIR}/pixel_othello_baseline'),
]):
    entropy_path = os.path.join(outdir, 'entropy_history.npy')
    if os.path.exists(entropy_path):
        entropies = np.load(entropy_path)
        ax.hist(entropies, bins=50, edgecolor='black', alpha=0.7)
        ax.axvline(x=0.5, color='r', linestyle='--', label='Deterministic threshold')
        ax.set_xlabel('Entropy (bits)')
        ax.set_ylabel('Count')
        ax.legend()
    ax.set_title(name)

plt.tight_layout()
plt.savefig(f'{OUT_DIR}/entropy_comparison.png', dpi=150)
plt.show()
print(f'Saved: {OUT_DIR}/entropy_comparison.png')

---
## Interpretation Guide

### What Success Looks Like

**2048 (Grid or Pixel):**
- Entropy histogram should be **bimodal**
- Peak near 0: deterministic slide/merge transitions
- Peak at ~3-4 bits: stochastic tile spawns

**Othello (Pixel):**
- Entropy histogram should be **unimodal near 0**
- Almost all transitions should have entropy < 0.5 bits

### What Failure Looks Like

- **High uniform entropy**: Model hasn't learned rules
- **All near-zero entropy for 2048**: Model collapsed (ignoring stochasticity)
- **High entropy for Othello**: Model hasn't learned deterministic rules

### Next Steps

1. If grid works but pixel fails: perception is the bottleneck
2. If both fail: architecture or training hyperparameters need tuning
3. If both succeed: ready for macro caching experiments!